In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import xesmf as xe
from metpy.units import units
from metpy import calc as mpcalc
import netCDF4
from netCDF4 import Dataset
from datetime import datetime, date, timedelta

In [2]:
real1_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202101_t.nc')
real2_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202102_t.nc')

In [3]:
#Regridding ERA5 data using xesmf

ds_out = xr.Dataset(
    {
        "lon": (["lon"], np.arange(0, 360, 1.5)),
        "lat": (["lat"], np.arange(-90, 91.5, 1.5)),
    }
)

regridder = xe.Regridder(real1_ds, ds_out, "bilinear")
regridder

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_401x1440_121x240.nc 
Reuse pre-computed weights? False 
Input grid shape:           (401, 1440) 
Output grid shape:          (121, 240) 
Periodic in longitude?      False

In [4]:
real1_rgrd = regridder(real1_ds)
real2_rgrd = regridder(real2_ds)

/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


In [5]:
# Vertical level specificaton
vlevel = 1000
levelStr = str(vlevel)

#Box coordinates: box2 = [[40,-103], [40,-93], [30,-93], [30,-103],[40,-103]]
# box3 = [[50,-104], [50,-93], [40,-93], [40,-104],[50,-104]]
# box4 = [[47,-103], [47,-92], [38,-92], [38,-103],[47,-103]]
lat_min = 30
lat_max = 41
lon_min = 257
lon_max = 267

In [6]:
rt1_rd.sel(lat=slice(lat_max,lat_min))

NameError: name 'rt1_rd' is not defined

In [ ]:
rt1_rd.sel(lon=slice(lon_min,lon_max))


In [7]:
#Reducing the datasets
r1 = real1_rgrd.reindex(lat=list(reversed(real1_rgrd.lat)))
t = r1['t'].sel(level=vlevel)#<--forecasted temps at 1000mb
rt1_rd = t.sel(time=t.time.dt.hour == 0)
rt1_rdf = rt1_rd.sel(lat=slice(lat_max,lat_min), lon=slice(lon_min,lon_max))


r2 = real2_rgrd.reindex(lat=list(reversed(real2_rgrd.lat)))
t2 = r2['t'].sel(level=vlevel)#<--forecasted temps at 1000mb
rt2_rd = t2.sel(time=t2.time.dt.hour == 0)
rt2_rdf = rt2_rd.sel(lat=slice(lat_max,lat_min), lon=slice(lon_min,lon_max))

In [8]:
rt1_rdf

<xarray.DataArray 't' (time: 31, lat: 8, lon: 7)>
array([[[283.1282959 , 281.44699097, 278.86602783, ..., 275.31143188,
         272.45526123, 271.91177368],
        [284.57373047, 282.43218994, 279.46731567, ..., 275.28137207,
         272.95016479, 272.58938599],
        [284.37945557, 280.94512939, 277.83224487, ..., 275.20040894,
         274.13427734, 273.68328857],
        ...,
        [279.38638306, 278.37341309, 275.23742676, ..., 278.93539429,
         279.01403809, 279.59683228],
        [276.4515686 , 275.47793579, 275.25360107, ..., 281.30822754,
         287.75137329, 290.13806152],
        [277.7906189 , 275.93585205, 276.55795288, ..., 280.28369141,
         286.04458618, 290.88967896]],

       [[282.8114624 , 281.19259644, 278.72033691, ..., 270.75543213,
         270.50335693, 270.02462769],
        [282.94790649, 280.79016113, 279.18978882, ..., 275.38079834,
         272.41595459, 271.97421265],
        [284.43264771, 282.65652466, 280.13568115, ..., 274.74020386,
         273.5838623 , 273.71334839],
...
        [296.25973511, 294.06732178, 291.56729126, ..., 286.60656738,
         286.19262695, 286.37994385],
        [292.28192139, 291.70373535, 292.55249023, ..., 288.7897644 ,
         287.2194519 , 286.73608398],
        [292.18939209, 290.7416687 , 291.16949463, ..., 288.70187378,
         285.33459473, 285.51269531]],

       [[285.70001221, 276.95574951, 275.30218506, ..., 275.63751221,
         277.21707153, 277.52233887],
        [287.83694458, 280.18426514, 276.87017822, ..., 279.6569519 ,
         282.8253479 , 281.41461182],
        [291.61355591, 285.19122314, 278.15603638, ..., 283.73422241,
         286.69213867, 286.00067139],
        ...,
        [294.89294434, 295.16583252, 295.02938843, ..., 294.47433472,
         292.81152344, 290.68615723],
        [296.67834473, 296.16491699, 296.03540039, ..., 295.8850708 ,
         292.08071899, 292.11077881],
        [300.4803772 , 298.79675293, 298.78979492, ..., 294.3795166 ,
         292.3767395 , 291.77081299]]])
Coordinates:
  * lat      (lat) float64 40.5 39.0 37.5 36.0 34.5 33.0 31.5 30.0
    level    int32 1000
  * time     (time) datetime64[ns] 2021-01-01 2021-01-02 ... 2021-01-31
  * lon      (lon) float64 258.0 259.5 261.0 262.5 264.0 265.5 267.0

In [9]:
rt2_rdf

<xarray.DataArray 't' (time: 28, lat: 8, lon: 7)>
array([[[286.84503174, 283.16271973, 278.51940918, ..., 272.01470947,
         272.32830811, 272.23526001],
        [287.26989746, 283.18093872, 277.44506836, ..., 273.44717407,
         273.45120239, 274.16137695],
        [288.77722168, 285.76461792, 280.62762451, ..., 276.01870728,
         275.140625  , 274.45675659],
        ...,
        [291.09381104, 288.51013184, 286.17532349, ..., 283.66650391,
         283.21734619, 283.17892456],
        [291.36087036, 289.91223145, 287.8404541 , ..., 285.2507019 ,
         285.14550781, 284.09545898],
        [293.59249878, 292.05484009, 291.16867065, ..., 287.88293457,
         287.8465271 , 286.91177368]],

       [[290.11053467, 281.88201904, 274.34750366, ..., 269.95303345,
         269.79522705, 269.10125732],
        [290.90969849, 288.95526123, 284.16827393, ..., 277.47946167,
         273.13153076, 271.29241943],
        [294.25613403, 291.26983643, 284.86022949, ..., 279.97613525,
         276.74301147, 273.82147217],
...
        [297.52770996, 295.62179565, 292.2935791 , ..., 285.60073853,
         284.20266724, 288.3664856 ],
        [298.65664673, 298.27832031, 294.14483643, ..., 289.4307251 ,
         295.18682861, 295.57122803],
        [295.36282349, 296.64758301, 290.40185547, ..., 295.83425903,
         292.99768066, 292.55459595]],

       [[284.83190918, 288.25115967, 288.769104  , ..., 288.7286377 ,
         286.50512695, 285.96490479],
        [287.33261108, 289.5824585 , 294.57983398, ..., 290.55563354,
         288.70639038, 287.95980835],
        [296.39468384, 298.66677856, 293.69165039, ..., 289.00177002,
         288.89251709, 285.77877808],
        ...,
        [302.29443359, 300.52209473, 295.55908203, ..., 294.32492065,
         294.97033691, 294.79229736],
        [301.19177246, 299.28182983, 298.77603149, ..., 296.72042847,
         295.04922485, 295.56921387],
        [297.72393799, 297.19589233, 298.15692139, ..., 295.76544189,
         293.46099854, 293.57632446]]])
Coordinates:
  * lat      (lat) float64 40.5 39.0 37.5 36.0 34.5 33.0 31.5 30.0
    level    int32 1000
  * time     (time) datetime64[ns] 2021-02-01 2021-02-02 ... 2021-02-28
  * lon      (lon) float64 258.0 259.5 261.0 262.5 264.0 265.5 267.0

In [10]:
rt1_ds = rt1_rdf.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/box_proj/box2_data/rd_t/realt_jan.nc",
                               mode='w')
rt2_ds = rt2_rdf.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/box_proj/box2_data/rd_t/realt_feb.nc",
                               mode='w')

In [11]:
#rt1_ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/box_proj/box4_data/rd_t/realt_jan.nc")
#rt2_ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/box_proj/box4_data/rd_t/realt_feb.nc")

In [12]:
#rt1_ds = rt1_ds.where(rt1_ds.t == 0, 'nan')
rt1_llmn = rt1_rdf.mean(dim=['lat','lon'])
rt2_llmn = rt2_rdf.mean(dim=['lat', 'lon'])

In [14]:
rt2_llmn

<xarray.DataArray 't' (time: 28)>
array([282.86882237, 284.31474904, 287.87553079, 292.44222423,
       284.37493896, 284.8587434 , 278.65099335, 279.48817689,
       277.35905457, 274.14622988, 271.46544538, 268.89915139,
       265.83724049, 266.18866866, 260.46353122, 260.77572359,
       266.03530066, 269.8476508 , 270.67013223, 277.94679969,
       284.18511745, 286.5392189 , 291.75320544, 295.74901744,
       288.09624754, 283.80720466, 288.82710321, 293.1491988 ])
Coordinates:
    level    int32 1000
  * time     (time) datetime64[ns] 2021-02-01 2021-02-02 ... 2021-02-28

In [15]:
rt1_llmn = rt1_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/box_proj/box2_data/llmn_t/realt_jan.nc",
                               mode='w')
rt2_llmn = rt2_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/box_proj/box2_data/llmn_t/realt_feb.nc",
                               mode='w')